In [1]:
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client()

In [3]:
NYC_TAXI_DATA_PARQUET = '/home/spotter/Documents/job/coiled_assessment/data/parquet/nyc-taxi'

In [4]:
use_cols = [
    'tpep_pickup_datetime',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'total_amount']
df = dd.read_parquet(NYC_TAXI_DATA_PARQUET, columns=use_cols)
df.head()

,tpep_pickup_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount
0,2019-01-01 00:46:40,1.0,1.5,7.0,1.65,9.95
1,2019-01-01 00:59:47,1.0,2.6,14.0,1.00,16.30
7,2019-01-01 00:21:28,1.0,1.3,6.5,1.25,9.05
8,2019-01-01 00:32:01,1.0,3.7,13.5,3.70,18.50
9,2019-01-01 00:57:32,2.0,2.1,10.0,1.70,13.00


In [5]:
df['tip_frac'] = df['tip_amount'] / df['total_amount']
df['pickup_hour'] = dd.to_datetime(df['tpep_pickup_datetime']).dt.hour
df.head()

,tpep_pickup_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,tip_frac,pickup_hour
0,2019-01-01 00:46:40,1.0,1.5,7.0,1.65,9.95,0.165829,0
1,2019-01-01 00:59:47,1.0,2.6,14.0,1.00,16.30,0.061350,0
7,2019-01-01 00:21:28,1.0,1.3,6.5,1.25,9.05,0.138122,0
8,2019-01-01 00:32:01,1.0,3.7,13.5,3.70,18.50,0.200000,0
9,2019-01-01 00:57:32,2.0,2.1,10.0,1.70,13.00,0.130769,0


In [6]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
